In [1]:
import qrml
import numpy        as np
import matplotlib.pyplot as plt

%matplotlib qt

In [2]:
n_points = 1000
np.random.seed(1)

theta = np.random.uniform(low=0, high=2*np.pi, size=n_points)
x = np.cos(theta)
y = np.sin(theta)
z = np.random.uniform(low=-3, high=3, size=n_points)

cylinder = np.stack([x, y, z], axis=-1)

In [16]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter3D(cylinder[:, 0], cylinder[:, 1], cylinder[:, 2], c=cylinder[:, 2])
plt.show()
fig.savefig("/home/lz1919/Documents/UNI/year_three/summer_tda/code/quotient_rml/images/cylinder.png", )

In [3]:
params = {'k':10, 'threshold_var':0.08, 'edge_sen':1, 'k0':100}

S = qrml.Simplex()
S.build_simplex(cylinder, **params)
S.normal_coords(**params)

Restricted license - for non-production use only - expires 2023-10-25


In [4]:
S.show_boundary(alpha=1, tol=2, c=cylinder[:, 2], show_pointcloud=True, **params)

No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [12]:
_ = S.plot_quotient(c=cylinder[:, 2], alpha=1, tol=2, quotient_tol=15, tol1=5, connection_tol=5, show_pointcloud=True)